In [1]:
from rdflib import Graph, URIRef, Namespace
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from tqdm import tqdm
import pandas as pd

In [72]:
#rdflib_dbpedia_graph = Graph("SPARQLStore")
#rdflib_dbpedia_graph.open("http://localhost:9999/bigdata/sparql")

graph = Graph()
graph.parse("/Users/yamamotobikutorueiichi/codes/i-align/i-align/DWY-NB/DW-NB/kg2.ttl")

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#gYear, Converter=<function parse_date at 0x10785a670>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0546'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#gYear, Converter=<function parse_date at 0x10785a670>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_fun

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#gYear, Converter=<function parse_date at 0x10785a670>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-400'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#gYear, Converter=<function parse_date at 0x10785a670>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func

<Graph identifier=Nb70e18f1b6bc43bcbe3461136cd5af51 (<class 'rdflib.graph.Graph'>)>

In [73]:
query = """
SELECT DISTINCT ?p
WHERE {
  ?s ?p ?o.
}"""
pred_res_list = graph.query(query)

In [74]:
count_total_query = """
SELECT (COUNT(*) AS ?totalCount)
WHERE {
    ?s <:pred> ?o
}
"""

count_total_obj_query = """
SELECT (COUNT(DISTINCT ?o) AS ?totalCount)
WHERE {
    ?s <:pred> ?o
}
"""

inv_func_dict = dict()
for pred_res in tqdm(pred_res_list):
    query_pred = pred_res[0]

    try:
        replaced_count_total_query = count_total_query.replace(':pred', str(query_pred))
        count_res = graph.query(replaced_count_total_query)
        total_count = 0
        for res in count_res:
            total_count = res.totalCount

        replaced_count_total_obj_query = count_total_obj_query.replace(':pred', str(query_pred))
        count_res = graph.query(replaced_count_total_obj_query)
        total_obj = 0
        for res in count_res:
            total_obj = res.totalCount
        
        inv_func_dict[str(query_pred)] = float(total_obj)/float(total_count)
    except Exception as e:
        print(e)
        print("exception raised: " + str(query_pred))

100%|█████████████████████████████████████████| 703/703 [00:08<00:00, 81.45it/s]


In [75]:
inv_func_list = list()
for key, value in inv_func_dict.items():
    inv_func_list.append({
        'predicate': key,
        'inverse_functionality': value
    })

In [76]:
inv_func_df = pd.DataFrame(inv_func_list)

In [77]:
inv_func_df

,predicate,inverse_functionality
0,http://www.wikidata.org/entity/P213,1.000000
1,http://www.wikidata.org/entity/P25,0.748656
2,http://www.w3.org/2000/01/rdf-schema#label,0.843023
3,http://www.wikidata.org/entity/P50,0.313776
4,http://www.wikidata.org/entity/P19,0.185189
...,...,...
698,http://www.wikidata.org/entity/P2080,1.000000
699,http://www.wikidata.org/entity/P1789,1.000000
700,http://www.wikidata.org/entity/P968,1.000000
701,http://www.wikidata.org/entity/P1989,1.000000


In [78]:
inv_func_df[inv_func_df['inverse_functionality'] > 0.5]

,predicate,inverse_functionality
0,http://www.wikidata.org/entity/P213,1.000000
1,http://www.wikidata.org/entity/P25,0.748656
2,http://www.w3.org/2000/01/rdf-schema#label,0.843023
6,http://www.wikidata.org/entity/P155,0.973815
8,http://www.wikidata.org/entity/P22,0.748628
...,...,...
698,http://www.wikidata.org/entity/P2080,1.000000
699,http://www.wikidata.org/entity/P1789,1.000000
700,http://www.wikidata.org/entity/P968,1.000000
701,http://www.wikidata.org/entity/P1989,1.000000


In [79]:
query = """
SELECT DISTINCT ?p
WHERE {
  ?s ?p ?o.
}"""
pred_res_list = graph.query(query)

In [80]:
count_total_query = """
SELECT (COUNT(*) AS ?totalCount)
WHERE {
    ?s <:pred> ?o
}
"""

count_total_subj_query = """
SELECT (COUNT(DISTINCT ?s) AS ?totalCount)
WHERE {
    ?s <:pred> ?o
}
"""

func_dict = dict()
for pred_res in tqdm(pred_res_list):
    query_pred = pred_res[0]

    try:
        replaced_count_total_query = count_total_query.replace(':pred', str(query_pred))
        count_res = graph.query(replaced_count_total_query)
        total_count = 0
        for res in count_res:
            total_count = res.totalCount

        replaced_count_total_subj_query = count_total_subj_query.replace(':pred', str(query_pred))
        count_res = graph.query(replaced_count_total_subj_query)
        total_obj = 0
        for res in count_res:
            total_obj = res.totalCount
        
        func_dict[str(query_pred)] = float(total_obj)/float(total_count)
    except Exception as e:
        print(e)
        print("exception raised: " + str(query_pred))

100%|█████████████████████████████████████████| 703/703 [00:08<00:00, 80.60it/s]


In [81]:
func_list = list()
for key, value in func_dict.items():
    func_list.append({
        'predicate': key,
        'functionality': value
    })

In [82]:
func_df = pd.DataFrame(func_list)

In [83]:
func_df

,predicate,functionality
0,http://www.wikidata.org/entity/P213,1.000000
1,http://www.wikidata.org/entity/P25,0.999587
2,http://www.w3.org/2000/01/rdf-schema#label,1.000000
3,http://www.wikidata.org/entity/P50,0.941327
4,http://www.wikidata.org/entity/P19,0.997708
...,...,...
698,http://www.wikidata.org/entity/P2080,1.000000
699,http://www.wikidata.org/entity/P1789,1.000000
700,http://www.wikidata.org/entity/P968,1.000000
701,http://www.wikidata.org/entity/P1989,1.000000


In [84]:
func_df[func_df['functionality'] > 0.5]

,predicate,functionality
0,http://www.wikidata.org/entity/P213,1.000000
1,http://www.wikidata.org/entity/P25,0.999587
2,http://www.w3.org/2000/01/rdf-schema#label,1.000000
3,http://www.wikidata.org/entity/P50,0.941327
4,http://www.wikidata.org/entity/P19,0.997708
...,...,...
698,http://www.wikidata.org/entity/P2080,1.000000
699,http://www.wikidata.org/entity/P1789,1.000000
700,http://www.wikidata.org/entity/P968,1.000000
701,http://www.wikidata.org/entity/P1989,1.000000


In [85]:
inversability_df = pd.merge(func_df, inv_func_df, on='predicate')
inversability_df['inversability'] = inversability_df['inverse_functionality'] * inversability_df['functionality']


In [86]:
inversability_df

,predicate,functionality,inverse_functionality,inversability
0,http://www.wikidata.org/entity/P213,1.000000,1.000000,1.000000
1,http://www.wikidata.org/entity/P25,0.999587,0.748656,0.748347
2,http://www.w3.org/2000/01/rdf-schema#label,1.000000,0.843023,0.843023
3,http://www.wikidata.org/entity/P50,0.941327,0.313776,0.295365
4,http://www.wikidata.org/entity/P19,0.997708,0.185189,0.184764
...,...,...,...,...
698,http://www.wikidata.org/entity/P2080,1.000000,1.000000,1.000000
699,http://www.wikidata.org/entity/P1789,1.000000,1.000000,1.000000
700,http://www.wikidata.org/entity/P968,1.000000,1.000000,1.000000
701,http://www.wikidata.org/entity/P1989,1.000000,1.000000,1.000000


In [87]:
inversability_df[inversability_df['inverse_functionality'] > 0.5]

,predicate,functionality,inverse_functionality,inversability
0,http://www.wikidata.org/entity/P213,1.000000,1.000000,1.000000
1,http://www.wikidata.org/entity/P25,0.999587,0.748656,0.748347
2,http://www.w3.org/2000/01/rdf-schema#label,1.000000,0.843023,0.843023
6,http://www.wikidata.org/entity/P155,0.980798,0.973815,0.955116
8,http://www.wikidata.org/entity/P22,1.000000,0.748628,0.748628
...,...,...,...,...
698,http://www.wikidata.org/entity/P2080,1.000000,1.000000,1.000000
699,http://www.wikidata.org/entity/P1789,1.000000,1.000000,1.000000
700,http://www.wikidata.org/entity/P968,1.000000,1.000000,1.000000
701,http://www.wikidata.org/entity/P1989,1.000000,1.000000,1.000000


In [88]:
inversability_df.to_csv('/Users/yamamotobikutorueiichi/codes/i-align/i-align/DWY-NB/DW-NB/kg2_inversability.csv')